In [0]:
%%capture
!pip install torch_geometric
!pip install torch_sparse
!pip install torch_scatter
!pip install torch_cluster
!pip install torch-spline-conv


In [2]:
!git clone https://github.com/leonardocunha2107/Link-Prediction
!rm -rf node_information
!mv Link-Prediction/* .
!rm -rf Link-Prediction

num_nodes=33226

Cloning into 'Link-Prediction'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 47 (delta 17), reused 38 (delta 8), pack-reused 0
Unpacking objects: 100% (47/47), done.


In [0]:
import torch
import numpy as np
import random

def edge_tensors(val_cut=0.8,device=torch.device("cuda")):
    neg_edges=[]
    pos_edges=[]
    ypos,yval=[]
    with open("training.txt", "r") as f:
        for line in f:
            line = line.split()
            edge=[int(line[0]),int(line[1])]
            if line[2] == '1':
                pos_edges.append(edge)
            else: neg_edges.append(edge)
    random.shuffle(pos_edges)
    random.shuffle(neg_edges)
    pos_edges,neg_edges=np.array(pos_edges),np.array(neg_edges)
    postrain_edges,negtrain_edges=pos_edges[:int(len(pos_edges)*val_cut)],neg_edges[:int(len(neg_edges)*val_cut)]
    posval_edges,negval_edges=pos_edges[int(len(pos_edges)*val_cut):],neg_edges[int(len(neg_edges)*val_cut):]
    val_edges=np.vstack([posval_edges,negval_edges])
    
    edge1,edge2=np.hstack([postrain_edges[:,0],postrain_edges[:,1]]),np.hstack([postrain_edges[:,1],postrain_edges[:,0]])
    postrain_edges_tensor=torch.tensor(np.vstack([edge1,edge2]),device=device,dtype=torch.long)
    
    edge1,edge2=np.hstack([negtrain_edges[:,0],negtrain_edges[:,1]]),np.hstack([negtrain_edges[:,1],negtrain_edges[:,0]])
    negtrain_edges_tensor=torch.tensor(np.vstack([edge1,edge2]),device=device,dtype=torch.long)
    
    edge1,edge2=np.hstack([val_edges[:,0],val_edges[:,1]]),np.hstack([val_edges[:,1],val_edges[:,0]])
    valedges_tensor=torch.tensor(np.vstack([edge1,edge2]),device=device,dtype=torch.long)
    
    ytrain=torch.tensor([1]*(postrain_edges_tensor.shape[1]))
    return postrain_edges_tensor,\
            torch.cat([postrain_edges_tensor,negtrain_edges_tensor],axis=1),valedges_tensor

n2v_tensor,train_data,val_data=edge_tensors()

In [5]:
print(n2v_tensor.shape)
print(val_tensor.shape)

torch.Size([2, 453796])
torch.Size([2, 181520])


In [7]:
import torch
#from data import  get_edge_tensor
from torch_geometric.nn.models import Node2Vec
#from node2vec import Node2Vec
device  = torch.device("cuda")
d=128
num_nodes=33226
n2v=Node2Vec(num_nodes,d,5,3,walks_per_node=3).to(device)
#n2v=Node2Vec(num_nodes,d,5,3,'text_embeds.npy',walks_per_node=2).to(device)
optimizer=torch.optim.Adam(n2v.parameters(),lr=1e-1)
n2v_tensor=n2v_tensor.to(device)
#print(edges_tensor[:,:10])
for i in range(100):
    
    loss=n2v.loss(n2v_tensor)
    optimizer.zero_grad()
    loss.backward(retain_graph=False)
    optimizer.step()
    print(f"Loss {loss.data}")
    

Loss 9.863016128540039
Loss 8.520421981811523
Loss 7.550008296966553
Loss 6.841428756713867
Loss 6.244897842407227
Loss 5.7738237380981445
Loss 5.383397579193115
Loss 5.014370441436768
Loss 4.716630935668945
Loss 4.450439929962158
Loss 4.177728176116943
Loss 3.9507694244384766
Loss 3.753190755844116
Loss 3.5460922718048096
Loss 3.367779493331909
Loss 3.193480968475342
Loss 3.0441927909851074
Loss 2.90509295463562
Loss 2.7736263275146484
Loss 2.6626944541931152
Loss 2.553676128387451
Loss 2.4445512294769287
Loss 2.3356692790985107
Loss 2.242884397506714
Loss 2.1573426723480225
Loss 2.0761501789093018
Loss 2.000098943710327
Loss 1.9217042922973633
Loss 1.8603825569152832
Loss 1.7968628406524658
Loss 1.7407807111740112
Loss 1.6881300210952759
Loss 1.6309468746185303
Loss 1.5865185260772705
Loss 1.54324471950531
Loss 1.5006697177886963
Loss 1.4627289772033691
Loss 1.4317281246185303
Loss 1.3913064002990723
Loss 1.3660449981689453
Loss 1.337429404258728
Loss 1.304802656173706
Loss 1.2832257

In [0]:
class  TrainDS:
    def __init__(self,embed):
        self.embed=embed
        self.edges=[]
        self.y=[]
        with open('training.txt','r') as fd:
            for l in fd:
                l=[int(x) for x in l.split()]
                self.edges.append((l[0],l[1]))
                self.y.append(l[2])
    def __len__(self):
        return len(self.y)
    def __getitem__(self,idx):
        edge=self.edges[idx]
        x=torch.cat([self.embed[edge[0]],self.embed[edge[1]]])
        return x,torch.tensor(self.y[idx])

In [0]:
#from data import TrainDS
from torch.utils.data.sampler import SubsetRandomSampler

embed=n2v(torch.arange(num_nodes,device=device)).data
"""dataset=TrainDS(embed)
dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(0.2 * dataset_size))
random_seed=42
np.random.seed(random_seed)
np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]


# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)
"""
batch_size=128
train_loader = torch.utils.data.DataLoader(train_tensor, batch_size=batch_size, 
                                           sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(val_tensor,dataset, batch_size=batch_size,
                                                sampler=valid_sampler)



In [0]:
from pprint import pprint


def train_epoch(model,optim,train_dl,loss_fn=nn.CrossEntropyLoss()):
    loss_avg=0
    device=torch.device("cuda")
    model.train()
    i=0
    for x,y in train_dl:
        optim.zero_grad()
        x = x.to(device=device, dtype=torch.float)  
        y = y.to(device=device, dtype=torch.long)


        scores = model(x)
        
        loss = loss_fn(scores, y)
        loss_avg+=loss.data
        loss.backward(retain_graph=False)

        optim.step()
        i+=1

    
    print('Loss %.4f' %loss/i)
    
    return loss_avg
def loader_stats(model,dataloader,threshold=0.5):
    tp,fp,tn,fn=0,0,0,0
    with torch.no_grad():
        model.eval()  # set model to evaluation mode
        
        for x,y in dataloader:
            x = x.to(device=device, dtype=torch.float)  
            y = y.to(device=device, dtype=torch.long)


            scores = model(x)
            scores=torch.nn.functional(softmax(scores))
            preds = (scores[:,1]>threshold).long()
            comp=(preds==y).long()

            ##calcule standard stats
            tp+=(comp*y).sum().data
            fp+=((1-comp)*(1-y)).sum().data
            tn+=((comp)*(1-y)).sum().data
            fn+=((1-comp)*y).sum().data
    tp,fp,tn,fn=float(tp)+1e-5,float(fp)+1e-5,float(tn)+1e-5,float(fn)+1e-5
    n=tp+fp+tn+fn
    raw={'n':n,'tp':tp,'fp':fp,'tn':tn,'fn':fn}

    pr,prec=tp/(tp+fn),tp/(tp+fp)
    metrics={'acc':(tp+tn)/n,'pr':pr,'prec':prec,'nr':tn/(tn+fp),'f1':(2*prec*pr)/(prec+pr)}  

    log={'raw':raw,'metrics':metrics}
    pprint(log['metrics'])
    return log


In [0]:
import torch.nn.functional as F
class Classifier(nn.Module):
    def  __init__(self,d):
        super(Classifier, self).__init__()
        self.fc1=nn.Linear(2*d,2*d)
        self.bn1=nn.BatchNorm1d(2*d)
        self.fc2=nn.Linear(2*d,2*d)
        self.fc3=nn.Linear(2*d,2*d)
        self.bn2=nn.BatchNorm1d(2*d)
        self.fc4=nn.Linear(2*d,2*d)
        self.fc5=nn.Linear(2*d,2)
    def forward(self,x):
        x=F.relu(self.fc1(x))
        x=self.bn1(x)
        x=F.relu(self.fc2(x))#+x
        x=F.relu(self.fc3(x))
        x=F.relu(self.fc4(x))+x
        x=self.bn2(x)
        x=F.relu(self.fc4(x))
        return self.fc5(x)




In [0]:
from sklearn.linear_model import LogisticRegression
import utils
classifier=nn.Sequential(nn.Linear(2*d,2*d),nn.ReLU(), nn.Linear(2*d,d),nn.ReLU(),nn.Linear(d,2)).cuda()
#classifier=Classifier(d).to(device=device)
optim=torch.optim.RMSprop(classifier.parameters(),lr=1e-4)
#weight=torch.tensor([1,5],device=device).type(torch.float)

for epoch in range(8):
    print(f'Epoch {epoch}')
    train_epoch(classifier,optim,train_loader,loss_fn=nn.CrossEntropyLoss())#weight=weight))
    _=loader_stats(classifier,train_loader)
    _=loader_stats(classifier,validation_loader)



Epoch 0
Loss 0.1684
{'acc': 0.9404387419013504,
 'f1': 0.9509222922645543,
 'nr': 0.9689058288030279,
 'pr': 0.9233529185816298,
 'prec': 0.980188661543339}
{'acc': 0.9366343831745019,
 'f1': 0.9477794221511937,
 'nr': 0.9650660155467603,
 'pr': 0.9195975470751347,
 'prec': 0.9777432225573773}
Epoch 1
Loss 0.0728
{'acc': 0.9704879378499783,
 'f1': 0.976121811258798,
 'nr': 0.9791946653037529,
 'pr': 0.9652621974192152,
 'prec': 0.987228556057106}
{'acc': 0.962483059327457,
 'f1': 0.9695925127141365,
 'nr': 0.9723586317685425,
 'pr': 0.9565654072256253,
 'prec': 0.9829793396407629}
Epoch 2
Loss 0.0197
{'acc': 0.9815225953777265,
 'f1': 0.9851454233390339,
 'nr': 0.9833146062580046,
 'pr': 0.9804470381423411,
 'prec': 0.9898890554820376}
{'acc': 0.9712865939592605,
 'f1': 0.9768746115975243,
 'nr': 0.9736524830336973,
 'pr': 0.9698689031329754,
 'prec': 0.9839822659072608}
Epoch 3
Loss 0.0306
{'acc': 0.9863430273980858,
 'f1': 0.9890280119422475,
 'nr': 0.9886389506284726,
 'pr': 0.98496

In [0]:
import csv
def generate_csv(model,embeds,filename='test.csv',thr=0.5):
    predictions = []
    with open("testing.txt", "r") as f:
        for l in f:
            l = [int(x) for x in l.split()]
            x=torch.cat([embeds[l[0]],embeds[l[1]]])
            x=model(x)[1]
            if x>thr:
                predictions.append("1")
            else: 
                predictions.append("0")
            
    
    predictions = zip(range(len(predictions)), predictions)
    # Write the output in the format required by Kaggle
    with open(filename,"w") as pred:
        csv_out = csv.writer(pred)
        csv_out.writerow(['id','predicted'])
        for row in predictions:
            csv_out.writerow(row) 
generate_csv(classifier,embed)

classifier=nn.Sequential(nn.Linear(2*d,2*d),nn.ReLU(), nn.Linear(2*d,d),nn.ReLU(),nn.Linear(d,2)).cuda()

optim=torch.optim.Adam(classifier.parameters(),lr=1e-5)

optim=torch.optim.Adam(classifier.parameters(),lr=1e-5)